<a href="https://colab.research.google.com/github/ssvadla/Demonstration/blob/main/Semi_Supervised_RF_unlabeled.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')

train1 = pd.read_csv('/content/drive/My Drive/Research/train_data1.csv')
train2 = pd.read_csv('/content/drive/My Drive/Research/train_data2.csv')
train3 = pd.read_csv('/content/drive/My Drive/Research/train_data3.csv')
train4 = pd.read_csv('/content/drive/My Drive/Research/train_data4.csv')
train5 = pd.read_csv('/content/drive/My Drive/Research/train_data5.csv')
train6 = pd.read_csv('/content/drive/My Drive/Research/train_data6.csv')
train7 = pd.read_csv('/content/drive/My Drive/Research/train_data7.csv')
train8 = pd.read_csv('/content/drive/My Drive/Research/train_data8.csv')
train9 = pd.read_csv('/content/drive/My Drive/Research/train_data9.csv')
train10 = pd.read_csv('/content/drive/My Drive/Research/train_data10.csv')
train_highKappa = pd.read_csv('/content/drive/My Drive/Research/train_data_highkappa.csv')
train1.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Unnamed: 0,Sentence,Target
0,659,Appellant had stated to the officers that she ...,Invalid
1,3456,We shall discuss the facts more fully in conne...,Others
2,2043,"â€œPerjury is a false statement, either writte...",Invalid
3,3344,The offense is felony theft by false pretext; ...,Issue
4,3231,Numerous contentions urging the commission of ...,Issue


In [2]:
unlabel_size = 5000
Threshold= 0.90

In [3]:
train = train1
train_list = [train2,train3,train4,train5,train6,train7,train8,train9,train10,train_highKappa]
for i in train_list:
  print(i)
  train = train.append(i)

      Unnamed: 0                                           Sentence    Target
0            659  Appellant had stated to the officers that she ...   Invalid
1           3456  We shall discuss the facts more fully in conne...    Others
2           2043  â€œPerjury is a false statement, either writte...   Invalid
3           3344  The offense is felony theft by false pretext; ...     Issue
4           3231  Numerous contentions urging the commission of ...     Issue
...          ...                                                ...       ...
3471        1561  Therefore, the deletion of reference to the Cl...  Analysis
3472         327  A paraffin test, which was made on the appella...  Analysis
3473        1462  The sole question presented is the sufficiency...     Facts
3474        1169  The answer was in response to the question ask...     Facts
3475        1470  The Stateâ€™s evidence shows that in the sprin...   Invalid

[3476 rows x 3 columns]
      Unnamed: 0                       

In [4]:
train.sort_values("Sentence", inplace = True)
print(len(train))

37711


In [5]:
 train = train.drop_duplicates(subset ="Sentence")

In [6]:
train['Target'].unique()


array(['Invalid', 'Rule/Law/Holding', 'Facts', 'Analysis', 'Others',
       'Conclusion', 'Issue'], dtype=object)

In [7]:
train['Target']=train['Target'].replace(['Others'],'Invalid')
train['Target'].unique()

array(['Invalid', 'Rule/Law/Holding', 'Facts', 'Analysis', 'Conclusion',
       'Issue'], dtype=object)

In [8]:
#cleaning
import nltk
import re
import string
nltk.download('stopwords')
nltk.download('wordnet')
stopword=nltk.corpus.stopwords.words('english')
from nltk.stem import WordNetLemmatizer
wl= WordNetLemmatizer()

def clean_text(text):
  text="".join([word.lower() for word in text if word not in string.punctuation])
  tokens = re.split('\W+',text)
  text = [wl.lemmatize(word) for word in tokens if word not in stopword]
  return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(analyzer = clean_text)
X_tfidf = tfidf_vect.fit_transform(train['Sentence'])
print(X_tfidf.shape)

(4416, 7374)


In [10]:
test = pd.read_csv(r'/content/drive/My Drive/Research/test_data.csv')
test['Target']=test['Target'].replace(['Others'],'Invalid')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
test['Sentence'] = test['Sentence'].str.replace('[^\w\s]','')
from nltk.corpus import stopwords
words = stopwords.words('english')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x for x in x.split() if x not in words))
t_p = tfidf_vect.transform(test['Sentence'])

In [11]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report



X, y = make_classification(random_state=1)

X_train, x_val, Y_train, y_val = train_test_split(X_tfidf,train['Target'],test_size=0.26,random_state=42)

classifier = RandomForestClassifier(n_estimators = 350, criterion = 'gini', max_features = 'auto', random_state = 42)
classifier.fit(X_train, Y_train)
y_pred = classifier.predict(x_val)
#print(np.unique(y_pred))
#print(type(y_pred))

Accuracy_score = accuracy_score(y_pred,y_val)

print('Validation Accuracy_score: %f' % Accuracy_score)
test_pred = classifier.predict(t_p)
acc_test = accuracy_score(test_pred,test['Target'])
print('Accuracy test data', acc_test)
classification_report_supervised = classification_report(test_pred,test['Target'], digits = 4)

print(classification_report_supervised)
matrix = confusion_matrix(y_pred, y_val)
print(matrix)


Validation Accuracy_score: 0.632724
Accuracy test data 0.6142034548944337
                  precision    recall  f1-score   support

        Analysis     0.1948    0.4839    0.2778        31
      Conclusion     0.5385    0.7778    0.6364        18
           Facts     0.8577    0.6580    0.7447       348
         Invalid     0.5301    0.4632    0.4944        95
           Issue     0.1765    0.7500    0.2857         8
Rule/Law/Holding     0.3529    0.5714    0.4364        21

        accuracy                         0.6142       521
       macro avg     0.4417    0.6174    0.4792       521
    weighted avg     0.7167    0.6142    0.6481       521

[[ 45  14  16   6   6   6]
 [  3  19   3   3   1   2]
 [113  17 526  67  35  38]
 [ 14   8  35  87   4  13]
 [  1   2   0   0  21   1]
 [  2   2   4   1   5  29]]


In [12]:
unlabel = pd.read_csv(r'/content/drive/My Drive/Research/Unlabeled_data.csv')
unlabel.head()

,Unnamed: 0,text,Complete
0,0,No motion for rehearing filed.,1
1,1,— Relator is under accusation of killing her h...,1
2,2,From an order of the District Judge of the 39t...,1
3,3,Relator and her husband had been married for a...,1
4,4,The evidence indicates that their family relat...,1


In [13]:
del unlabel['Complete']
del unlabel['Unnamed: 0']

In [14]:
unlabel['text'] = unlabel['text'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
unlabel['text'] = unlabel['text'].str.replace('[^\w\s]','')
from nltk.corpus import stopwords
words = stopwords.words('english')
unlabel['text'] = unlabel['text'].apply(lambda x: " ".join(x for x in x.split() if x not in words))


In [15]:
from textblob import TextBlob
from textblob import Word
nltk.download('wordnet')
nltk.download('punkt')
unlabel['text'] = unlabel['text'].apply(lambda x: TextBlob(x).words)
unlabel['text'] = unlabel['text'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x]))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [16]:
unlabel_1 = unlabel.loc[:unlabel_size]

In [17]:
def index_reset(unlabel_2):
  unlabel_2.reset_index(inplace=True)
  del unlabel_2['index']
  print(unlabel_2.head())
  return unlabel_2

In [18]:
unlabel_1 = index_reset(unlabel_1)

                                                text
0                             motion rehearing filed
1    relator accusation killing husband virgil clark
2  order district judge 39th judicial district re...
3                relator husband married number year
4  evidence indicates family relation always plea...


In [19]:
x_un1 = tfidf_vect.transform(unlabel_1['text'])

In [20]:
x_un1.shape
pred_unlabel_1 = classifier.predict_proba(x_un1)
pred_unlabel_1



array([[0.05379252, 0.26285714, 0.33672109, 0.33805782, 0.00285714,
        0.00571429],
       [0.        , 0.00571429, 0.32857143, 0.62571429, 0.03428571,
        0.00571429],
       [0.08285714, 0.02285714, 0.52      , 0.26857143, 0.03714286,
        0.06857143],
       ...,
       [0.1       , 0.        , 0.85428571, 0.02571429, 0.01428571,
        0.00571429],
       [0.02857143, 0.00857143, 0.88571429, 0.05428571, 0.01142857,
        0.01142857],
       [0.15714286, 0.04857143, 0.48      , 0.30285714, 0.        ,
        0.01142857]])

In [21]:
x_un1.shape

(5001, 7374)

In [22]:
import numpy as np
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_1:
  if max(j)> Threshold:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(type(pos))
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 1, 1, 2, 2, 1, 1, 3, 1, 2, 3, 2, 2, 2, 2, 2, 2, 1, 3, 1, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 3, 1, 2, 3, 1, 2, 2, 1, 3, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 3, 3, 2, 2, 2, 3, 1, 1, 2, 2, 2, 2, 3, 3, 3, 1, 2, 3, 2, 3, 3, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 1, 1, 2, 3, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 2, 2, 2, 2, 3, 2, 3, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 3, 3, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 1, 2, 1, 2, 3, 3, 3, 3, 3, 2, 2, 2, 3, 3, 2, 2, 2, 1, 2, 2, 1, 1, 2]
[0.9428571428571428, 0.9142857142857143, 0.905, 0.9744897959183673, 0.9285714285714286, 0.9142857142857143, 0.9107142857142856, 0.9792857142857143, 0.9514285714285714, 0.9307142857142856, 0.9342857142857143, 0.9428571428571428, 0.9114285714285715, 0.9285714285714286, 0.925714

In [23]:
unlabel_1 = unlabel_1.loc[pos,:]
train_data_size = len(unlabel_1)
print(len(unlabel_1))
unlabel_1.head()

241


,text
6,brown slept one bed deceased daughter went bed...
7,short time retired child left deceased bed got...
10,judgment therefore affirmed
63,reason stated judgment reversed cause remanded
69,three people house homicide occurred deceased ...


In [24]:
class_x_un1 = tfidf_vect.transform(unlabel_1['text'])

In [25]:
class_x_un1.shape

(241, 7374)

In [26]:

class_pred_unlabel_1 = classifier.predict(class_x_un1)
class_pred_unlabel_1


array(['Facts', 'Facts', 'Conclusion', 'Conclusion', 'Facts', 'Facts',
       'Conclusion', 'Conclusion', 'Invalid', 'Conclusion', 'Facts',
       'Invalid', 'Facts', 'Facts', 'Facts', 'Facts', 'Facts', 'Facts',
       'Conclusion', 'Invalid', 'Conclusion', 'Invalid', 'Invalid',
       'Facts', 'Facts', 'Facts', 'Facts', 'Facts', 'Facts', 'Facts',
       'Facts', 'Facts', 'Facts', 'Facts', 'Facts', 'Facts', 'Facts',
       'Facts', 'Facts', 'Facts', 'Facts', 'Facts', 'Facts', 'Conclusion',
       'Facts', 'Invalid', 'Conclusion', 'Facts', 'Invalid', 'Conclusion',
       'Facts', 'Facts', 'Conclusion', 'Invalid', 'Facts', 'Facts',
       'Conclusion', 'Facts', 'Facts', 'Facts', 'Facts', 'Facts', 'Facts',
       'Facts', 'Conclusion', 'Facts', 'Facts', 'Facts', 'Conclusion',
       'Facts', 'Invalid', 'Invalid', 'Facts', 'Facts', 'Facts',
       'Invalid', 'Conclusion', 'Conclusion', 'Facts', 'Facts', 'Facts',
       'Facts', 'Invalid', 'Invalid', 'Invalid', 'Conclusion', 'Facts',
      

In [27]:
unlabel_1['Target']=class_pred_unlabel_1
unlabel_1.head()

,text,Target
6,brown slept one bed deceased daughter went bed...,Facts
7,short time retired child left deceased bed got...,Facts
10,judgment therefore affirmed,Conclusion
63,reason stated judgment reversed cause remanded,Conclusion
69,three people house homicide occurred deceased ...,Facts


In [28]:
from sklearn.metrics import classification_report
from sklearn import svm

x_train_1 = tfidf_vect.transform(unlabel_1['text'])
x_train_1.shape
X_train, x_val, Y_train, y_val = train_test_split(x_train_1,unlabel_1['Target'],test_size=0.25,random_state=2)
classifier_1 = RandomForestClassifier(n_estimators = 350, criterion = 'gini', max_features = 'auto', random_state = 42)
classifier_1.fit(X_train, Y_train)
y_pred = classifier_1.predict(x_val)
Accuracy_score = accuracy_score(y_pred,y_val)

print('Validation ------- Accuracy_score: %f' % Accuracy_score)

test_pred = classifier_1.predict(t_p)
acc_test = accuracy_score(test_pred,test['Target'])
print('Test ------Accuracy score ', acc_test)
matrix = confusion_matrix(y_pred, y_val)
print(matrix)
print("---------validation--------------")
print(classification_report(y_val,y_pred,digits=4))
print("----------test---------------------")
classification_report_semi_supervised = classification_report(test['Target'],test_pred, digits=4)
print(classification_report_semi_supervised)

Validation ------- Accuracy_score: 0.983607
Test ------Accuracy score  0.418426103646833
[[ 8  0  0]
 [ 0 46  0]
 [ 0  1  6]]
---------validation--------------
              precision    recall  f1-score   support

  Conclusion     1.0000    1.0000    1.0000         8
       Facts     1.0000    0.9787    0.9892        47
     Invalid     0.8571    1.0000    0.9231         6

    accuracy                         0.9836        61
   macro avg     0.9524    0.9929    0.9708        61
weighted avg     0.9859    0.9836    0.9841        61

----------test---------------------
                  precision    recall  f1-score   support

        Analysis     0.0000    0.0000    0.0000        77
      Conclusion     0.6250    0.3846    0.4762        26
           Facts     0.6604    0.5243    0.5846       267
         Invalid     0.2321    0.8193    0.3617        83
           Issue     0.0000    0.0000    0.0000        34
Rule/Law/Holding     0.0000    0.0000    0.0000        34

        accurac

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [29]:

# Write line to file

with open('/content/drive/My Drive/Research/Results/Example2.txt', 'a') as writefile:
  writefile.write("\n")
  writefile.write(" RF ")
  writefile.write("\n")
  writefile.write("Threshold = ")
  writefile.write(str(Threshold))
  writefile.write("\n")
  writefile.write("unlabel size  = ")
  writefile.write(str(unlabel_size))
  writefile.write("\n")
  writefile.write("train data size  = ")
  writefile.write(str(train_data_size))
  writefile.write("\n")
  writefile.write("Supervised Resutls on Test Data")
  writefile.write("\n")
  writefile.write(classification_report_supervised)
  writefile.write("\n")
  writefile.write("Semi Supervised Resutls on Test Data")
  writefile.write("\n")
  writefile.write(classification_report_semi_supervised)